In [1]:
# ! python3 -m pip install natasha

# **Named Entity Recognition (NER)**

Воспользуемся готовыми инструментами для выделения именованных сущностей

Создадим базовый пайплаин для NER
- Воспользуемя библиотекой `natasha` и используем регулярные выражение `re`


## <div style="padding: 30px;color:white;margin:0px;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#3B454F"><b><span style='color:#FFFFFF'>1 |</span></b> <b>Библиотека natasha</b></div>

### **1.1. Импортируем модуль**

- Одна из библиотек которые можно использовать для выделения именованных сущностей это библиотека `natasha`
- `natasha` это готовый инструмент для `NER`, работает только для русского языка
- Посмотрим как этой библиотекой пользоваться для выделения именованных сущностей

In [34]:
import os
import re
from tqdm import tqdm

# загружаем все компоненты которые нам понадобятся
from natasha import (Segmenter,MorphVocab,
                     NewsEmbedding,NewsMorphTagger,NewsSyntaxParser,NewsNERTagger,
                     PER,NamesExtractor,DatesExtractor,MoneyExtractor,AddrExtractor,
                     Doc
                    )

Инициализируем все наши компоненты

In [35]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
# dates_extractor = DatesExtractor(morph_vocab)
# money_extractor = MoneyExtractor(morph_vocab)
# addr_extractor = AddrExtractor(morph_vocab)

### **1.2. Чтение данных**

Читаем данные

In [36]:
with open('res.txt', 'r') as f:
    text = f.read().strip()

In [37]:
text

'Предположение о том, что мы произошли от обезьяны в 21 веке терпит крах. Ученые зашли в тупик. За годы исследований им так и не удалось доказать теорию эволюции и найти общих предков человека и приматов. Иначе современные шимпанзе или гориллы уже давно превратились бы в людей. Мы слишком не похожи друг на друга, чтобы быть родственниками. И даже генетики это подтверждают. Так что Дарвин был неправ. Никакого отношения человек к обезьянам не имеет. До сих пор ученые, биологи, палеонтологи, антропологи и прочие, и типа меня персонажи, вынуждены это опровергать. И поэтому нынче я, Станислав Дорбышевский, буду жарить этот дурацкий миф вместе с Рбака Трендами. На планете существует очень много млекопитающих. И эти млекопитающие по-разному родственны друг другу. Есть мерзуны, рукокрылые, хищные, копытные, китообразные, панголины и всякие прочие. А есть приматы. Приматов много разных. И удивительным образом, человек это просто один из великого множества этих самых приматов. И это родство, есл

In [38]:
len(text)

13899

### **1.3. `natasha` document**

Иницилизируем класс которые будет хранить все данные о тексте, в отличие от `spacy`, doc это не готовая pipeline

Нам надо будет вызывать отделные методы для того чтобы заполнить содержание которое нас интересует:
- `.segmenter` - токенизация, разбиваем на предложение
- `.tag_morph` - морфологический разбор токенов (e.g. NOUN)
- `.tag_ner` - выделение именованных сущностей

In [39]:
doc = Doc(text)
doc

Doc(text='Предположение о том, что мы произошли от обезьяны...)

In [65]:
[a for a in dir(doc) if not a.startswith('_')]

['as_json',
 'clear_envelopes',
 'envelop_sent_spans',
 'envelop_sent_tokens',
 'envelop_span_tokens',
 'from_json',
 'morph',
 'ner',
 'parse_syntax',
 'segment',
 'sents',
 'spans',
 'syntax',
 'tag_morph',
 'tag_ner',
 'text',
 'tokens']

In [40]:
# vars(doc)

### **1.4. Разбиваем документ на предложения и токены**

Разбиваем текста на токены (`tokens`) и предложения (`sents`), данные сохраняются в `doc`

In [41]:
# Добавляем в doc tokens, sents
doc.segment(segmenter)

display(doc) # документ
display('sentences',doc.sents[:3]) # предложения
display('tokens',doc.tokens[:5])  # токены

Doc(text='Предположение о том, что мы произошли от обезьяны..., tokens=[...], sents=[...])

'sentences'

[DocSent(stop=72, text='Предположение о том, что мы произошли от обезьяны..., tokens=[...]),
 DocSent(start=73, stop=94, text='Ученые зашли в тупик.', tokens=[...]),
 DocSent(start=95, stop=203, text='За годы исследований им так и не удалось доказать..., tokens=[...])]

'tokens'

[DocToken(stop=13, text='Предположение'),
 DocToken(start=14, stop=15, text='о'),
 DocToken(start=16, stop=19, text='том'),
 DocToken(start=19, stop=20, text=','),
 DocToken(start=21, stop=24, text='что')]

### **1.5. Выделяем часть речи**

Вызовим метод `tag_morph` и передадим объект `morph_tagger`
- Сделаем морфологический разбор документа
- "POS" - часть речи

In [42]:
# выделяем части речи
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)
display(doc.tokens[:15])

[DocToken(stop=13, text='Предположение', id='1_1', head_id='1_13', rel='nsubj', pos='NOUN', feats=<Inan,Acc,Neut,Sing>),
 DocToken(start=14, stop=15, text='о', id='1_2', head_id='1_3', rel='case', pos='ADP'),
 DocToken(start=16, stop=19, text='том', id='1_3', head_id='1_1', rel='nmod', pos='PRON', feats=<Inan,Loc,Neut,Sing>),
 DocToken(start=19, stop=20, text=',', id='1_4', head_id='1_7', rel='punct', pos='PUNCT'),
 DocToken(start=21, stop=24, text='что', id='1_5', head_id='1_7', rel='mark', pos='SCONJ'),
 DocToken(start=25, stop=27, text='мы', id='1_6', head_id='1_7', rel='nsubj', pos='PRON', feats=<Nom,Plur,1>),
 DocToken(start=28, stop=37, text='произошли', id='1_7', head_id='1_3', rel='acl', pos='VERB', feats=<Perf,Ind,Plur,Past,Fin,Act>),
 DocToken(start=38, stop=40, text='от', id='1_8', head_id='1_9', rel='case', pos='ADP'),
 DocToken(start=41, stop=49, text='обезьяны', id='1_9', head_id='1_7', rel='obl', pos='NOUN', feats=<Anim,Gen,Fem,Sing>),
 DocToken(start=50, stop=51, text='

### **1.6. Выделяем именнованые сущности**
- Для выделения именованных сущностей в документе вызываем `tag_ner`
- Именованные сущности сохраняются в `.spans`
    - В `spans.type` сохраняется класс именованной сущности
    - В `spans.text` сохраняется выделенная именнованая сущность

In [43]:
doc.tag_ner(ner_tagger)
len(doc.spans)

8

In [44]:
doc.spans

[DocSpan(start=383, stop=389, type='PER', text='Дарвин', tokens=[...]),
 DocSpan(start=584, stop=606, type='PER', text='Станислав Дорбышевский', tokens=[...]),
 DocSpan(start=647, stop=661, type='PER', text='Рбака Трендами', tokens=[...]),
 DocSpan(start=7655, stop=7663, type='PER', text='Поедение', tokens=[...]),
 DocSpan(start=9141, stop=9153, type='PER', text='Пургаториуса', tokens=[...]),
 DocSpan(start=9349, stop=9361, type='PER', text='Пургаториуса', tokens=[...]),
 DocSpan(start=11547, stop=11553, type='LOC', text='Африке', tokens=[...]),
 DocSpan(start=12972, stop=12981, type='LOC', text='Австралии', tokens=[...])]

In [46]:
set([s.text for s in doc.spans])

{'Австралии',
 'Африке',
 'Дарвин',
 'Поедение',
 'Пургаториуса',
 'Рбака Трендами',
 'Станислав Дорбышевский'}

In [47]:
for span in doc.spans:
    print(span.text,span.type)

Дарвин PER
Станислав Дорбышевский PER
Рбака Трендами PER
Поедение PER
Пургаториуса PER
Пургаториуса PER
Африке LOC
Австралии LOC


Можно лемматизировать содержание используя `.normalize`, данные сохраняются в `span.normal`

In [48]:
# лемматизируем извлеченные сущности
for span in doc.spans:
    span.normalize(morph_vocab)

{s.text: s.normal for s in doc.spans}

{'Дарвин': 'Дарвин',
 'Станислав Дорбышевский': 'Станислав Дорбышевский',
 'Рбака Трендами': 'Рбака Тренды',
 'Поедение': 'Поедение',
 'Пургаториуса': 'Пургаториус',
 'Африке': 'Африка',
 'Австралии': 'Австралия'}

In [49]:
# извлечем персон
for span in doc.spans:
    if(span.type == PER):
        span.extract_fact(names_extractor)

{s.normal: s.fact.as_dict for s in doc.spans if s.fact}

{'Дарвин': {'last': 'Дарвин'},
 'Станислав Дорбышевский': {'first': 'Станислав', 'last': 'Дорбышевский'},
 'Рбака Тренды': {'last': 'Рбака'}}

## <div style="padding: 30px;color:white;margin:0px;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#3B454F"><b><span style='color:#FFFFFF'>2 |</span></b> <b>Объединенный метод NER</b></div>


### **2.1. Подготовка данных**

`natasha` может упускать именованные сущности, мы можем использовать результаты из `re`

In [50]:
# лемматизация
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from functools import lru_cache

In [51]:
dataset_folder = '.'
files = os.listdir(dataset_folder)
files = [i for i in files if '.txt' in i]
files

['res.txt', 'res2.txt']

In [52]:
texts = []
for f in files:
    with open(os.path.join(dataset_folder, f), 'r') as fo:
        texts.append(fo.read())

### **2.2. Вспомогательные функции**

(1) **NER** с `natasha`

Вспомогательная функция для NER с `natasha`

In [53]:
def get_ner_natasha(text):
    text = re.sub(r'[^s\d\w\-:,\.\?\!]', ' ', text) # убираем пунктуация
    doc = Doc(text)                                 # создаем natasha документ
    doc.segment(segmenter)                          # tokenise, sentences
    doc.tag_ner(ner_tagger)                         # tag NER

    # normalise using morpher
    for span in doc.spans:
        span.normalize(morph_vocab)

    res = set((s.normal for s in doc.spans))
    return res

(2) **NER** с `re`

Воспомогательная функция для NER с `re`, воспользуемся регулярками и лемматизатора из `pymorphy2`

In [54]:
@lru_cache(10000)
def lemmatize(s):
    s = str(s).lower()
    return morph.parse(s)[0].normal_form.capitalize()

In [55]:
reg1 = re.compile(r'[^s\d\w\-:,\.\?\!]')
reg2 = re.compile(r'([\.\?!])')

def get_ner_regex(s):
    s = reg1.sub(' ', s)
    s = reg2.sub(r'\g<1><sep>', s)

    # разбиваем на предложения
    sent1 = [sent.strip() for sent in s.split('<sep>')]
    sent2 = [' '.join(ss.split()[1:]) for ss in sent1]

    res = []
    for ss in sent2:
        res.extend([e.strip() for e in re.findall(r'(?:[A-ZА-ЯЁ][A-ZА-ЯЁа-яёa-z\d-]+\s*)+', ss)])

    return set((lemmatize(s) for s in res))

(3) Объединим результаты из `natasha` с `re`

In [56]:
def get_ner(text):
    return get_ner_regex(text).union(get_ner_natasha(text))

### **2.3. Проверка подхода**

Подтвердим что все работает по отдельности
- `get_ner_natasha` нам возвращает сущности которые выделила `natasha`
- `get_ner_regex` нам возвращает сущности которые выделил `re`
- `get_ner` нам возващает объединение двух подходов

In [57]:
text = texts[1]
print(text)

Ракета-носитель «Союз-СТ-А» успешно запущена из Гвианского космического центра 26 апреля в 0:02 по московскому времени. На борту находились спутники Sentinel-1B, Microscope и FYS, сообщает Роскосмос.
Sentnel-1B предназначен для наблюдения за сушей и океанами в радиодиапазоне. Спутник идентичен запущенному двумя годами ранее Sentinel-1A, оба аппарата Sentinel-1 будут работать в паре, собирая данные с противоположных точек орбиты. Sentinel-1 помогают следить за состоянием океанов, ледников и лесов. Получаемые с орбиты данные используются для обнаружения айсбергов и нефтяных разливов, а также для предоставления актуальных картографических данных при чрезвычайных ситуациях. Кроме того, данные со спутников используются в различных «непрофильных» исследованиях. Например, группа немецких ученых опубликовала исследование о предполагаемом месте испытаний северокорейского ядерного и термоядерного оружия, в работе использовался и интерферометрический снимок предполагаемого места, сделанный Sentin

In [58]:
# NER который нашел natasha
get_ner_natasha(text)

{'FYS',
 'Гвианского космического центра',
 'Европейским космическим агентством',
 'Европейской комиссией',
 'Коперник',
 'Николай Воронцов',
 'Роскосмос'}

In [59]:
# NER который нашел re
get_ner_regex(text)

{'Fys',
 'Microscope',
 'Sentinel',
 'Sentinel-1',
 'Sentinel-1a',
 'Sentinel-1b',
 'Sentinel-3a',
 'Воронцов',
 'Гвианский',
 'Европейский',
 'Коперник',
 'Роскосмос',
 'Союз-ст-'}

In [60]:
# объединим подходы
get_ner(text)

{'FYS',
 'Fys',
 'Microscope',
 'Sentinel',
 'Sentinel-1',
 'Sentinel-1a',
 'Sentinel-1b',
 'Sentinel-3a',
 'Воронцов',
 'Гвианский',
 'Гвианского космического центра',
 'Европейский',
 'Европейским космическим агентством',
 'Европейской комиссией',
 'Коперник',
 'Николай Воронцов',
 'Роскосмос',
 'Союз-ст-'}

### **2.4. Работа на реальном примере**

Все нормально работает, теперь возмем наш основной корпус документов (отзывы о банке)

In [61]:
import pandas as pd

df_review = pd.read_csv('review_cleaned.csv',usecols=['user','review_cleaned'])
user_review = list(df_review['user'].values)
actual_review = list(df_review['review_cleaned'].values)

user_review = user_review[:50]
actual_review = actual_review[:50]

In [62]:
res_ners = []
for user, text in tqdm(zip(user_review,actual_review)):
    res_ners.append((user, sorted(get_ner(text))))

50it [00:02, 24.78it/s]


Пример NER для одного отзыва

In [77]:
res_ners[0][1][:10]

['MasterCard',
 'MasterCard Standard',
 'Mastercard',
 'Mastercard mass',
 'Mastercard standard',
 'Вы',
 'Евгении',
 'Евгений',
 'Евгения',
 'МИР']